# MNE 2 R - Instagram Influencers Study

Script to export summary measures in specified time windows, for each sensor/trial/subject, to text (CSV) files for import into R.

Summary measures available include:
- `mean`: mean amplitude in time window
- `aman`: adaptive mean negative amplitude in time window (mean over narrow time range, centred on minimum value in the larger time window)
- `amap`: adaptive mean positive amplitude
---
Copyright (c) 2018 Aaron J Newman, NeuroCognitive Imaging Lab, Dalhousie University

This program is free software; you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation; either version 2 of the License, or (at your option) any later version. This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

In [1]:
import mne
import numpy as np
import pandas as pd
from os import mkdir
import os, sys
import csv

# supress MNE's verbosity:
mne.set_log_level('error')

In [2]:
# all subjects; not separated by group

data_path = '.'

subjects = [
 'ID01',
    'ID02',
    'ID03',
    'ID04',
    'ID05',
    'ID06',
    'ID07',
    'ID08',
    'ID09',
    'ID10',
    'ID11',
    'ID12',
    'ID13',
    'ID14',
    'ID15',
    'ID16',
    'ID17',
    'ID18',
    'ID19',
    'ID20',
    'P00',
    'P01'
]

# Amount of time to shift event codes by, based on empirical testing with photocell
#  to determine lag between event code and actual stimulus appearance on screen.
#  For reasons that are unclear, we need to double this value (in sec) to get the correct shift
tshift = 0.169 

P3_timewin = (0.2, 0.4)
LPP_timewin = (0.6, 1.2) # LPP most pronounced btw. 600 - 1200ms after stimulus onset: https://www.frontiersin.org/articles/10.3389/fnhum.2012.00033/full
                         # 600-1000ms are also seen as middle window of the LPP -> https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3019134/   
N170_timewin = (0.13, 0.2)
N400_timewin = (0.3, 0.5)

# Define start & end of time window to compute mean amplitude over (depvar:'mean')
#   Adaptive mean amplitude (negative) depvar:'aman'
#   Adaptive mean amplitude (positive) depvar:'amap'
# note that here, we work in milliseconds whereas in other scripts MNE often works in sec
# dict of timewins of interest - will get one output file per subject per timewin
timewins = {
            'P3_mean': {'label':'P3', 'timewin': (200, 400), 'depvar':'mean'},
            'P3_peak_pos': {'label':'P3', 'timewin': (200, 400),'depvar':'amap'},
            'LPP_mean': {'label':'LPP', 'timewin': (600, 1200), 'depvar':'mean'}, 
            'LPP_peak_pos': {'label':'LPP', 'timewin': (600, 1200), 'depvar':'aman'},
            'N170_mean': {'label':'N170', 'timewin': (130, 200), 'depvar':'mean'},
            'N170_peak_neg': {'label':'N170', 'timewin': (130, 200),'depvar':'aman'},
            'N400_mean': {'label':'N400', 'timewin': (300, 500), 'depvar':'mean'},
            'N400_peak_neg': {'label':'N400', 'timewin': (300, 500),'depvar':'aman'},
            }
ama_width = 30  # width in ms around peak amplitude to compute adaptive mean amplitude

scaling_time = 1e3  # re-scale time to milliseconds

## Function for computing adaptive mean amplitude

Credit for this method of computation goes to Marijn van Vliet

https://gist.github.com/wmvanvliet/5cc013ef0f9b18561c74a4d6c1d130b7

In [3]:
def adaptive_mean(subject, epochs, tw_label, timewin, depvar, ama_width):
    epoch_start = timewin[0]
    epoch_end   = timewin[1] 
    # time win in which to find peak; +1 ensures python gets last time point in the series by 
    search_window = (float(epoch_start)/1000, (float(epoch_end)+1)/1000)  #to float; -Colin

    ind_start, ind_stop = np.searchsorted(epochs.times, search_window)

    #having trouble here. problem seems to be with the epoch_start variable
    data_to_search = epochs.get_data()[:, :, ind_start:ind_stop]
    
    if depvar=='aman':
        peaks = data_to_search.argmin(axis=2)
    elif depvar=='amap':
        peaks = data_to_search.argmax(axis=2)
    else: 
        print('Error: depvar must be aman or amap')

    # The indices are currently based on the data_to_search array. Add the index
    # of the start of the search window to the peaks to obtain indexes that can be
    # used with the original epochs data.
    peaks += ind_start

    # Centre on peak, ± half of ama_widy, in sec
    peak_window = (-ama_width/2/1000, ama_width/2/1000) 

    # Convert the peak window into an array of sample indices
    peak_window = np.arange(int(peak_window[0] * epochs.info['sfreq']),
                            int(peak_window[1] * epochs.info['sfreq']+1))

    # Use numpy broadcasting to add the peaks to the peak_window. This gives us for
    # each epoch and channel, the corresponding time window over which to compute
    # the mean.
    time_windows = peaks[:, :, np.newaxis] + peak_window[np.newaxis, np.newaxis, :]
    time_windows.shape
    
    # Use advanced integer indexing to get the corresponding data from the epochs
    n_epochs, n_channels, n_samples = epochs.get_data().shape
    peak_data = epochs.get_data()[
        np.arange(n_epochs)[:, np.newaxis, np.newaxis],  # All epochs
        np.arange(n_channels)[np.newaxis, :, np.newaxis],  # All channels
        time_windows  # Only the requested time windows
    ]
    
    means = peak_data.mean(axis=2)
    # convert MNE's default scaling (V) to microvolts
    means = means * 1e6

    # get condition labels for each epoch
    id_swapped = dict((v, k) for k, v in epochs.event_id.items())
    conditions = [id_swapped[k] for k in epochs.events[:, 2]]

    # number the epochs
    # use normal 1,... indexing for output rather than python's 0,...
    epoch_nums = np.arange(1, np.shape(epochs.events)[0]+1)

    # converting to DataFrame allows easy insertion of epoch numbers and condition labels, then CSV output
    df = pd.DataFrame(data=means, columns=epochs.ch_names, index=[epoch_nums, conditions])
    
    # Write to CSV
    out_fname = out_path + subject + '_' + tw_label + '_' + str(epoch_start) + '-' + str(epoch_end) + '.csv'
    df.to_csv(out_fname)

## Do the work, looping through subjects. 


In [4]:
for subject in subjects:
    # Make output directory
    out_path = data_path + '/EEG-Results/'
    if not os.path.exists(out_path):
        os.makedirs(out_path)

    # Read in the epoched, cleaned data    
    epochs_fname = data_path + '/Epochs/' + subject + '-epo.fif'
    epochs = mne.read_epochs(epochs_fname, verbose=None)   
    
    # -> add rereference to tp9 and tp10
    epochs = epochs.set_eeg_reference(ref_channels=['TP9', 'TP10'])

    
    epochs._raw_times = epochs._raw_times - tshift
    epochs._times_readonly = epochs._times_readonly - tshift
    
    
    # loop through each timewindow/component of interest, creating mean amplitudes then writing out
    for tw in timewins.keys():

        if timewins[tw]['depvar']=='mean':
            epoch_start = timewins[tw]['timewin'][0]
            epoch_end   = timewins[tw]['timewin'][1]
            time_win = np.arange(epoch_start,epoch_end)

            out_fname = out_path + subject + '_' + tw + '_' + str(epoch_start) + '-' + str(epoch_end) + '.csv'

            # Create pandas dataFrame from MNE epochs
            df = epochs.to_data_frame(index=['epoch', 'time', 'condition'])
            #df = epochs.to_data_frame(scaling_time=scaling_time, index=['epoch', 'time', 'condition'])

            # Create multiIndex; see http://pandas.pydata.org/pandas-docs/version/0.15.2/advanced.html#advanced-reindexing-and-alignment
            idx = pd.IndexSlice

            # Get mean ampl over specified time range for each trial, for magnetometers
            means = df.loc[idx[:,time_win],:].groupby(['epoch','condition']).mean()

            # Write to CSV
            means.to_csv(out_fname)
            
        elif timewins[tw]['depvar']=='aman' or timewins[tw]['depvar']=='amap':
            adaptive_mean(subject, epochs, tw, timewins[tw]['timewin'], timewins[tw]['depvar'], ama_width)